## 1. Можно ли отобрать наиболее значимые признаки с помощью PCA?

Метод главных компонент создает новые признаки, поэтому нет, нельзя.

## 2*. Примите участие в одном или двух соревнованиях


#### Псевдоним на Kaggle

Anna Taganova

#### Ноутбук по регрессии

https://colab.research.google.com/drive/11GenKRtNo7MXh2IycTNtl0opHqtC-67L

#### Ноутбук по классификации

пока не строится ROC-curv, но модель неплохо работает

https://colab.research.google.com/drive/1QJUVTQoGsapiGrXHxUXiaMNbruuqHWeY

In [1]:
import numpy as np
from sklearn.datasets import load_iris

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import warnings
warnings.simplefilter('ignore')

## 3*. Написать свою реализацию метода главных компонент с помощью сингулярного разложения с использованием функции numpy.linalg.svd()

In [2]:
X, y = load_iris(return_X_y=True)

In [3]:
scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)

In [4]:
U, D, V = np.linalg.svd(X_scaled)

In [5]:
# Мы получили сингулярные числа матрицы
D

array([20.92306556, 11.7091661 ,  4.69185798,  1.76273239])

In [6]:
eig_sum = sum(D)
var_exp = [(i / eig_sum) * 100 for i in D]
cum_var_exp = np.cumsum(var_exp)
print(f'Доля дисперсии, описвыаемая каждой из компонент \n{var_exp}')

# а теперя оценим кумулятивную (то есть накапливаемую) дисперсию при учитывании каждой из компонент
print(f'Кумулятивная доля дисперсии по компонентам \n{cum_var_exp}')

Доля дисперсии, описвыаемая каждой из компонент 
[53.52971788236239, 29.956812784020087, 12.00368241720948, 4.509786916408034]
Кумулятивная доля дисперсии по компонентам 
[ 53.52971788  83.48653067  95.49021308 100.        ]


Таким образом, оставив 3 признака, мы потеряем меньше 5% информации

In [7]:
# Создадим матрицу весов
W = V[:, :3]
W

array([[ 0.52106591, -0.26934744,  0.5804131 ],
       [-0.37741762, -0.92329566, -0.02449161],
       [ 0.71956635, -0.24438178, -0.14212637],
       [ 0.26128628, -0.12350962, -0.80144925]])

Получаем новую матрицу с тремя признаками

In [8]:
X_ = X @ W

## 4*. Обучить любую модель классификации на датасете iris до применения PCA и после него. Сравнить качество классификации по отложенной выборке.

#### Загрузка данных, разбиение и стандартизация

In [9]:
X, y = load_iris(return_X_y=True)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.3, random_state = 5
)

In [11]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#### Обучение модели до применения PCA

In [12]:
lr = LogisticRegression()

lr.fit(X_train, y_train)

y_pred_train = lr.predict(X_train)
y_pred_test = lr.predict(X_test)

print(accuracy_score(y_train, y_pred_train), accuracy_score(y_test, y_pred_test))

0.9523809523809523 0.9333333333333333


#### Обучение модели после применения PCA

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    X_, y, test_size = 0.3, random_state = 5
)

lr.fit(X_train, y_train)

y_pred_train = lr.predict(X_train)
y_pred_test = lr.predict(X_test)

print(accuracy_score(y_train, y_pred_train), accuracy_score(y_test, y_pred_test))

0.9428571428571428 0.9333333333333333


Возможно, мы уменьшили переобучение, но результаты очень похожи